<a href="https://colab.research.google.com/github/jiyanshud22/Champhunt-user-recommendation/blob/main/ball_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Import libraries
import cv2
import numpy as np
import os

# Path to the input video
video_path = '/content/in2.mp4'

# Path for the output video
output_video_path = '/content/processed_video.avi'

# Load video
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Function to check if an object is a ball based on circularity, size, and aspect ratio
def is_ball(contour):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    circularity = 4 * np.pi * (area / (perimeter * perimeter))

    # Define stricter size and circularity thresholds for the ball
    return 150 < area < 800 and 0.85 < circularity < 1.15

# Initialize a variable to store the previous ball's location for temporal consistency
previous_ball_center = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define lower and upper bounds for red color (commonly used for cricket balls)
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])

    # Create masks for red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = mask1 | mask2

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(red_mask, (7, 7), 0)

    # Find contours in the mask
    contours, _ = cv2.findContours(blurred, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected_ball = False  # Flag to indicate if a ball is detected in this frame

    # Loop over contours and filter for circularity and size
    for contour in contours:
        if is_ball(contour):
            # Get the bounding circle of the contour
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)

            # Temporal consistency check
            if previous_ball_center is not None:
                distance = np.linalg.norm(np.array(previous_ball_center) - np.array(center))
                if distance > 50:  # Ignore objects that are too far from the previous ball position
                    continue

            # Draw circle and label it as 'Ball'
            cv2.circle(frame, center, radius, (0, 255, 0), 2)
            cv2.putText(frame, "Cricket Ball", (int(x) - 10, int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)
            previous_ball_center = center
            detected_ball = True
            break  # Only consider the first valid ball detected in the frame

    # If no ball is detected, reset the previous ball center
    if not detected_ball:
        previous_ball_center = None

    # Save the current frame to the output video
    out.write(frame)

cap.release()
out.release()

print(f"Processed video saved at: {output_video_path}")


Processed video saved at: /content/processed_video.avi
